In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
files = sorted(glob('output-imda/*.json'), key = lambda x: int(x.split('-')[-1].replace('.json', '')))
len(files)

68448

In [10]:
import mp
import copy

minimum_score = 6

def loop(files):
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            results.append(data[i])

    return results

In [11]:
results = mp.multiprocessing(files, loop, cores = 50)

 85%|████████▌ | 1169/1368 [01:00<00:29,  6.64it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 1368/1368 [01:25<00:00, 15.91it/s]

 60%|██████    | 826/1368 [01:27<00:26, 20.53it/s]]


100%|██████████| 1368/1368 [01:27<00:00, 15.56it/s]

 74%|███████▎  | 1006/1368 [01:30<00:04, 80.58it/s]

100%|██████████| 48/48 [00:00<00:00, 3142.44it/s]


In [12]:
len(results)

2874528

In [13]:
results[0]

{'predict_ms': [50258,
  50282,
  50360,
  50365,
  430,
  1538,
  29216,
  7408,
  11,
  43365,
  3680,
  13877,
  992,
  3779,
  16281,
  717,
  289,
  657,
  282,
  12711,
  1706,
  66,
  12584,
  1706,
  73,
  9286,
  17289,
  74,
  892,
  449,
  7691,
  1988,
  23171,
  1026,
  40463,
  545,
  23059,
  569,
  5581,
  717,
  302,
  4579,
  3077,
  13,
  50715,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257],
 'predict_en': [50258,
  50259,
  50360,
  50365,
  4928,
  4104,
  82,
  365,
  3779,
  6352,
  645,
  14658,
  281,
  853,
  5145,
  641,
  1281,
  12126,
  2507,
  613,
  21688,
  4358,
  13,
  50715,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257],
 'score_ms': 10.375,
 'score_en': 8.75,
 'filename': 'IMDA-STT/part1-mp3/000010101.mp3'}

In [16]:
with open('prepared-imda-original.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 2874528/2874528 [00:49<00:00, 57661.58it/s]


In [17]:
from huggingface_hub import HfApi
api = HfApi()

In [18]:
api.upload_file(
    path_or_fileobj='prepared-pseudolabel-original.jsonl',
    path_in_repo='prepared-pseudolabel-original.jsonl',
    repo_id='mesolitica/pseudolabel-imda-large-v3-timestamp',
    repo_type='dataset',
    
)

prepared-pseudolabel-original.jsonl:   0%|          | 0.00/7.53G [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/pseudolabel-imda-large-v3-timestamp/blob/main/prepared-pseudolabel-original.jsonl'

In [24]:
import mp
import copy
import re

pattern_pair = r'<\|(\d+\.\d+)\|>(.*?)<\|(\d+\.\d+)\|>'

minimum_score = 6

def loop(files):
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            a = tokenizer._decode_asr(
                [{'tokens': np.array([data[i]['predict_en']])}], 
                return_timestamps = True, return_language = 'en', 
                time_precision = 0.02)[1]['chunks']
            
            a = [a_['text'] for a_ in a]

            results.extend(a)

    return results

In [32]:
results = mp.multiprocessing(files, loop, cores = 50)

In [26]:
len(results)

7033620

In [27]:
with open('imda-text.texts', 'w') as fopen:
    for t in tqdm(set(results)):
        if not len(t):
            continue
        
        fopen.write(f'{json.dumps(t)}\n')

100%|██████████| 4938979/4938979 [00:05<00:00, 842985.76it/s]


In [29]:
!ls -lh imda-text.texts

-rw-r--r-- 1 ubuntu ubuntu 247M Apr 13 04:29 imda-text.texts


In [31]:
api.upload_file(
    path_or_fileobj='imda-text.texts',
    path_in_repo='imda-text.texts',
    repo_id='mesolitica/pseudolabel-imda-large-v3-timestamp',
    repo_type='dataset',
)

imda-text.texts:   0%|          | 0.00/258M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/pseudolabel-imda-large-v3-timestamp/blob/main/imda-text.texts'

In [1]:
!head -n 10 imda-text.texts

" No, I think you're supposed to guess this word."
" I'm like so lucky"
" And like the boyfriend's time"
" And then the other thing, the challenging part that we are facing is because the meat"
" Gestures means a movement, a part of body, like to express an idea."
" tour goers are also very nice to hang out with."
" for BTO the least the HDB this for BTO"
" So, why have your name so that I can confirm your order?"
" My will to live"
" here's a good example of multiculturalism."
